In [2]:
import os
import math
import random
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch import utils
from mdl import torch as mdl

In [3]:
tb_data = pd.read_csv('/home/mikewei/tarball/UserBehavior.csv')
tb_data.columns = ['uid', 'iid', 'cid', 'act', 'ts']

print(tb_data)

              uid      iid      cid act          ts
0               1  2333346  2520771  pv  1511561733
1               1  2576651   149192  pv  1511572885
2               1  3830808  4181361  pv  1511593493
3               1  4365585  2520377  pv  1511596146
4               1  4606018  2735466  pv  1511616481
...           ...      ...      ...  ..         ...
100150801  999999  4797808    11120  pv  1512293403
100150802  999999  4613472  4602841  pv  1512293766
100150803  999999  3647364  2304296  pv  1512293792
100150804  999999  1903801  2304296  pv  1512293827
100150805  999999  3696094  4602841  pv  1512293891

[100150806 rows x 5 columns]


In [4]:
print(f'uid: {tb_data.uid.min(), tb_data.uid.max(), len(tb_data.uid.unique())}')
print(f'iid: {tb_data.iid.min(), tb_data.iid.max(), len(tb_data.iid.unique())}')
print(f'cid: {tb_data.cid.min(), tb_data.cid.max(), len(tb_data.cid.unique())}')
print(f'act: {tb_data.act.unique()}')

uid: (1, 1018011, 987994)
iid: (1, 5163070, 4162024)
cid: (80, 5162429, 9439)
act: ['pv' 'fav' 'buy' 'cart']


In [62]:
train_df = pd.read_csv('/home/data/taobao/tb_data_train.csv')
test_df = pd.read_csv('/home/data/taobao/tb_data_test.csv')
train_df, test_df

(             uid     iid   cid  pv  fav  cart  buy          ts  ulen      neg  \
 0              0       0     0   1    0     0    0  1511544070     0   553991   
 1              0       1     1   1    0     0    0  1511561733     1  2096907   
 2              0       2     2   1    0     0    0  1511572885     2  3290336   
 3              0       3     3   1    0     0    0  1511593493     3   261027   
 4              0       4     0   1    0     0    0  1511596146     4  2638749   
 ...          ...     ...   ...  ..  ...   ...  ...         ...   ...      ...   
 74983975  987993    7894   964   1    0     0    0  1512293252    36  2857250   
 74983976  987993  635418  2360   1    0     0    0  1512293282    37  2891616   
 74983977  987993  655781  2360   1    0     0    0  1512293403    38  1884042   
 74983978  987993  638297   706   1    0     0    0  1512293792    39   358038   
 74983979  987993   78548   706   1    0     0    0  1512293827    40  3054020   
 
           neg

In [134]:
class TaobaoData(mdl.DataModule):
    def __init__(self, batch_size=32):
        super().__init__()
        self.save_hyperparameters()
        self.train_df = train_df
        self.test_df = test_df
        self.train_ds = TaobaoData.Dataset(self, True)
        self.test_ds = TaobaoData.Dataset(self, False)
    class Dataset:
        def __init__(self, data, is_train=True):
            self.data = data
            self.is_train = is_train
            self.df = self.data.train_df if self.is_train else self.data.test_df 
        def __len__(self):
            return len(self.df)
        def __getitem__(self, idx):
            return (self.df.uid[idx], self.df.iid[idx], self.df.loc[idx, ['pv','fav','cart','buy']].to_numpy(), 
                    self.df.neg[idx], self.df.negc[idx])
data = TaobaoData()

In [136]:
loader = torch.utils.data.DataLoader(data.train_ds, batch_size=2)
next(iter(loader))

[tensor([0, 0]),
 tensor([0, 1]),
 tensor([[1, 0, 0, 0],
         [1, 0, 0, 0]]),
 tensor([ 553991, 2096907]),
 tensor([ 11, 671])]

In [128]:
# %timeit train_df.loc[234, ['pv','fav','cart','buy']].to_numpy()
# %timeit train_df.loc[344, 'pv']
# %timeit t = torch.tensor(arr2[2324, :])
t1 = (train_df.pv.to_numpy())
t2 = (train_df.fav.to_numpy())
t3 = (np.array([t1, t2]))
%timeit tt = torch.from_numpy(t3)

402 ns ± 1.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
